# Promptpal Example



In [15]:
!pip install -e ../.

Obtaining file:///Users/jdaniel/Source/promptpal
  Installing build dependencies ... one
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for promptpal (pyproject.toml) ... done
  Created wheel for promptpal: filename=promptpal-1.3.10-0.editable-py3-none-any.whl size=7510 sha256=7ffdd20e19146ebffeba5d47fffb01023943f58f6164427ea6addcb8e6a59f36
  Stored in directory: /private/var/folders/l6/4f11zktx3vdf5nljgxw9mbmc0000gn/T/pip-ephem-wheel-cache-mq0kp8rj/wheels/88/00/1c/c3cfcdf57be7575cb2dafda296b55518e134a43783da8d0559
Successfully built promptpal
  Attempting uninstall: promptpal
    Found existing installation: promptpal 1.3.10
    Uninstalling promptpal-1.3.10:
      Successfully uninstalled promptpal-1.3.10

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# Introduction

Promptpal has one main class that is used to interact with the utilities. 

In [1]:
# Import the Promptpal class and create an instance
from promptpal import Promptpal
pal = Promptpal()

In [2]:
# Show the available roles and what they are for
pal.list_roles()

['analyst',
 'assistant',
 'artist',
 'bioworker',
 'developer',
 'data_scientist',
 'data_visualization',
 'editor',
 'image',
 'photographer',
 'prompt_engineer',
 'refactor',
 'unit_tester',
 'writer',
 'chain_of_thought',
 'refine_prompt',
 'glyph_prompt',
 'summarizer',
 'computational_biologist',
 'educator']

# Example 1

Using the coder and unit_tester and educator roles to write code and help understand the code that was written.

In [3]:

# Use the refactor role to improve some code and document the changes
# code generated will be written to `generated_files`
request = """
The following python script
Rewrite the code to follow better practices and be as much more concise. 
Thoroughly document and comment any changes.

To be refactored:
```python

#!/usr/bin/python
'''USAGE: annotate_fasta.py BLASToutput Fasta
This script annotates a fasta file with sequence names determined from a BLAST of that file.
'''
import sys


# Write dictionary from BLAST output
with open(sys.argv[1],'r') as blast_output:
        blast_dictionary = {}
        gene_count = 0
        for line in blast_output:
                gene_count += 1
                current = str(gene_count).zfill(9)
                line = line.split()
                query = line[0]
                target = line[1]
                evalue = line[10]
                entry = target + '|evalue_' + str(evalue) + '|' + current
                blast_dictionary[query] = entry


# Parse the fasta file and print translated names and unchanged sequence data
with open(sys.argv[2],'r') as input_fasta:
        seq_count = 0
        annotated = 0
        key_errors = 0
        hypothetical = 0
        unknown = 0
        uncharacterized = 0
        outfile_name = str(sys.argv[2]).rstrip('fastn') + 'annotated.fasta'
        output_fasta = open(outfile_name, 'w')

        for line in input_fasta:

                if str(line)[0] == '>':
                        seq_count += 1
                        entry = str(line).strip('>').strip()
                        entry = entry.rstrip('\n')

                        try:
                                blast_hit = blast_dictionary[entry]
                                annotated += 1
                                if 'hypothetical' in blast_hit: hypothetical += 1
                                if 'unknown' in blast_hit: unknown += 1
                                if 'uncharacterized' in blast_hit: uncharacterized += 1
                        except KeyError:
                                key_errors += 1
                                blast_hit = entry + '|unknown:' + str(key_errors)

                        final_entry = '>' + blast_hit + '\n'
                        output_fasta.write(final_entry)
                        continue

                else:
                        output_fasta.write(line + '\n')

output_fasta.close()


# Write some stats to a file about the success of the annotation
logfile_name = str(sys.argv[2]).rstrip('fastn') + 'annotation_logfile.txt'
with open(logfile_name,'w') as logfile:
        logfile.write(str(seq_count) + ' total sequences.\n')
        logfile.write(str(key_errors) + ' sequences could not be annotated.\n')
        logfile.write(str(seq_count - key_errors) + ' sequences recieved some annotation.\n')
        logfile.write(str(hypothetical) + ' sequences annotated as hypothetical.\n')
        logfile.write(str(unknown) + ' sequences annotated as unknown function.\n')
        logfile.write(str(uncharacterized) + ' sequences annotated as uncharacterized.\n')
        logfile.write(str(annotated - hypothetical - unknown - uncharacterized) + ' sequences have an informative annotation.\n')
```
"""
pal.chat("refactor", request)

INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.


```python
#!/usr/bin/env python3

"""
Annotates a FASTA file with sequence names derived from a BLAST output.

This script parses a BLAST output file and a FASTA file.  It uses the BLAST
results to annotate the FASTA sequences, adding information about the
best BLAST hit to the sequence header.  Sequences without a BLAST hit
are marked as "unknown".

Usage:
    annotate_fasta.py <blast_output> <input_fasta>

Arguments:
    blast_output: Path to the BLAST output file (tabular format, e.g., -outfmt 6).
    input_fasta:  Path to the input FASTA file.

Output:
    A new FASTA file named <input_fasta>.annotated.fasta with the annotated sequences.
    A log file named <input_fasta>.annotation_logfile.txt containing annotation statistics.
"""

import sys
from pathlib import Path


def parse_blast_output(blast_file):
    """
    Parses the BLAST output file and creates a dictionary mapping query sequence
    names to annotation strings.

    Args:
        blast_file (str): Path to the BLAST ou

In [5]:
# Use the unit test role to add some tests. context is saved between chat messages.
pal.chat("unit_tester", "Generate unit test for the newly optimized code and explicitly document the purpose of each test.")

INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.
INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.
INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.


```python
import unittest
from unittest.mock import mock_open, patch
import sys  # Import sys for capturing stdout
from io import StringIO  # Import StringIO

# Assuming your script is named annotate_fasta.py and is in the same directory
from annotate_fasta import (parse_blast_output, annotate_fasta, write_output,
                             write_log_file, main)
from pathlib import Path


class TestAnnotateFasta(unittest.TestCase):

    def test_parse_blast_output_valid_file(self):
        """
        Test that parse_blast_output correctly parses a valid BLAST output file.
        Checks if the dictionary is created with the expected key-value pairs.
        """
        blast_content = """seq1 target1 other_data other_data other_data other_data other_data other_data other_data other_data 1e-5
seq2 target2 other_data other_data other_data other_data other_data other_data other_data other_data 1e-10"""
        with patch("builtins.open", mock_open(read_data=blast_content)):
           

In [6]:
# Utilize the educator role to help understand the code and tests
request = """
Create a short lesson plan for helping a beginner coder to master the python principles 
implemented in the refactored code and unit tests described previously.
"""
pal.chat("educator", request)

INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.


Okay, here's a short lesson plan designed to help a beginner coder understand the Python principles implemented in the refactored `annotate_fasta.py` code and its unit tests:

**Lesson Plan: Mastering Python Principles through FASTA Annotation**

**Goal:**  To guide a beginner coder in understanding and applying key Python principles used in the refactored `annotate_fasta.py` script and its unit tests, improving their code readability, maintainability, and testability.

**Prerequisites:**

*   Basic understanding of Python syntax (variables, loops, conditional statements).
*   Familiarity with the concept of functions.

**Duration:** ~4-6 hours (can be broken into multiple sessions)

**Materials:**

*   The refactored `annotate_fasta.py` script.
*   The unit test script (`test_annotate_fasta.py`).
*   Example FASTA and BLAST output files (for understanding the script's purpose).
*   A Python IDE (e.g., VS Code, PyCharm, Thonny).
*   Jupyter Notebook (optional, for interactive examples)

# Example 2

We can start a new chat thread and create a new role


In [7]:
pal.new_chat()

In [3]:
request = """
Write a system role for running logistics for a virology lab. 
The agent should be able to analyze a new wet lab protocol and create a list of necessary reagents and tools, 
then build a strategy for procurement of dispensable or non-reusable components. 
If possible, search online for providers with item or SKU numbers along with URLs to the product pages.
"""

# Setting write_output to false will write the output to a variable
response = pal.chat("prompt_engineer", request)

INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.


Okay, here's a system role description for an agent designed to handle logistics for a virology lab.  This role prioritizes accuracy, efficiency, and cost-effectiveness in reagent and tool procurement:

**System Role: Virology Lab Logistics Coordinator (VL-LogCon)**

**Description:**

You are VL-LogCon, an AI assistant specializing in the logistical operations of a virology wet lab. Your primary function is to ensure the lab has all necessary reagents, tools, and consumables, in the right quantities, at the right time, and within budget. You achieve this by analyzing experimental protocols, generating comprehensive procurement lists, and developing optimized procurement strategies. You are meticulously thorough, resourceful, and focused on reducing errors and minimizing downtime due to supply shortages.

**Goals:**

*   **Accurate Reagent and Tool Identification:** Translate wet lab protocols into complete and accurate lists of required reagents, tools, and consumables.
*   **Efficient

In [4]:
# We can use Promptpals prompt refinement tools to help make our new role prompt even better
# There are a few options for improvmenet, prompts, glyph_refinement, chain_of_thought, and keyword_refinement.
# we will use glyph_refinement
prompt = """
**System Role: Virology Lab Logistics & Procurement Agent**

**Description:**

You are an expert logistics and procurement agent specifically designed to support the efficient operation of a virology wet lab. Your primary responsibilities are to analyze experimental protocols, identify all necessary reagents and supplies (both disposable and non-disposable), create detailed procurement plans, identify potential vendors, and generate purchase orders. You are optimized for speed, accuracy, and cost-effectiveness.

**Goals:**

*   **Minimize experiment downtime** due to missing reagents or supplies.
*   **Optimize reagent and supply inventory levels** to avoid shortages and reduce waste/expiration.
*   **Secure the most cost-effective pricing** for all necessary items without compromising quality or reliability.
*   **Maintain a comprehensive and up-to-date database** of reagents, supplies, vendors, pricing, and product information.
*   **Streamline the procurement process** for lab personnel.
*   **Ensure compliance** with all relevant safety regulations and institutional policies regarding procurement and handling of biological materials.
*   **Identify potential alternatives** to expensive or difficult-to-obtain reagents.

**Skills and Capabilities:**

*   **Protocol Analysis:** Ability to meticulously analyze wet lab protocols (including but not limited to PCR, cell culture, ELISA, viral titration, transfection, and flow cytometry) to identify all required reagents, consumables, and equipment. This includes inferring necessary items that may be implicitly required but not explicitly stated in the protocol.
*   **Reagent and Supply Identification:**  Possess a deep understanding of common and specialized reagents, consumables, and equipment used in virology research.  This includes:
    *   Buffers and Solutions (e.g., PBS, Tris-HCl, cell culture media formulations, lysis buffers).
    *   Enzymes (e.g., reverse transcriptase, polymerases, restriction enzymes).
    *   Antibodies (primary and secondary, conjugated and unconjugated).
    *   Cell Culture Supplies (e.g., flasks, dishes, media, serum, antibiotics).
    *   PCR/qPCR Reagents (e.g., primers, dNTPs, master mixes).
    *   Viral Titration Supplies (e.g., TCID50 plates, plaque assay reagents).
    *   Consumables (e.g., pipette tips, microcentrifuge tubes, gloves, filters, serological pipettes).
    *   Specialized Equipment (e.g. real-time PCR machines, flow cytometers, centrifuges, incubators).
*   **Vendor Identification and Research:** Ability to search online and access internal databases to identify potential vendors for each required item.  This includes considering:
    *   **Price:** Obtain and compare pricing from multiple vendors.
    *   **Lead Time:**  Estimate delivery times and factor them into procurement plans.
    *   **Product Quality and Reliability:** Assess vendor reputation and product specifications.
    *   **Availability:** Verify stock levels and anticipate potential supply chain disruptions.
    *   **Shipping Costs:** Factor in shipping and handling fees.
    *   **Minimum Order Quantities:** Consider minimum order requirements.
    *   **Institutional Preferred Vendors:** Prioritize vendors with existing contracts or agreements with the institution.
*   **Online Search Expertise:**  Proficient in using search engines (Google, Google Scholar, etc.) and specialized scientific databases to locate product specifications, vendor information, and Material Safety Data Sheets (MSDS).  You should be able to identify and extract:
    *   Product Names
    *   Catalog/SKU Numbers
    *   Manufacturer Names
    *   URLs to Product Pages
    *   Pricing Information
    *   Technical Specifications
    *   Safety Information (MSDS)
*   **Procurement Strategy:** Develop a comprehensive procurement strategy that considers:
    *   **Inventory Levels:**  Determine optimal quantities of each item to order based on usage rates and lead times.
    *   **Batch Ordering:** Consolidate orders to reduce shipping costs and administrative overhead.
    *   **Standing Orders:** Establish standing orders for frequently used items.
    *   **Expiration Dates:** Prioritize ordering items with longer expiration dates.
    *   **Storage Requirements:** Consider storage conditions (e.g., temperature, light sensitivity) when planning orders.
    *   **Budget Constraints:** Adhere to allocated budgets for each experiment or project.
    *   **Alternative Sourcing:** Identify and evaluate alternative sources for critical reagents to mitigate supply chain risks.
*   **Purchase Order Generation:**  Create accurate and complete purchase orders that include:
    *   Item Descriptions
    *   Catalog/SKU Numbers
    *   Quantities
    *   Vendor Information
    *   Pricing
    *   Shipping Address
    *   Billing Address
    *   Required Delivery Date
*   **Communication:** Effectively communicate with lab personnel, vendors, and institutional procurement departments.
*   **Database Management:** Maintain a comprehensive database of reagents, supplies, vendors, pricing, and product information.  This database should be easily searchable and accessible to authorized lab personnel.
*   **Safety Compliance:**  Ensure all procurement activities comply with relevant safety regulations and institutional policies regarding the handling and storage of biological materials and chemicals.
*   **Reporting:** Generate reports on inventory levels, spending, and procurement activities.

**Constraints:**

*   You must adhere to the stated budget limitations for each project or experiment.
*   You must comply with all applicable institutional procurement policies and regulations.
*   You must prioritize safety and accuracy in all your activities.
*   You cannot order items that are explicitly prohibited by the institution or by law.
*   You should attempt to source items from vendors that are pre-approved by the institution whenever possible.
*   You must provide clear and concise explanations for all your recommendations and decisions.

**Input:**

*   A detailed wet lab protocol.
*   Budgetary information for the experiment or project.
*   Access to the lab's existing inventory database (if available).
*   Institutional procurement policies and procedures.
*   A list of preferred vendors (if available).

**Output:**

*   A detailed list of all required reagents, consumables, and equipment, including:
    *   Item Name
    *   Description
    *   Catalog/SKU Number
    *   Manufacturer
    *   Quantity
    *   Estimated Unit Price
    *   Estimated Total Price
    *   Vendor Name
    *   URL to Product Page (if available)
    *   Justification for including the item in the list.
*   A procurement strategy, including:
    *   Recommended order quantities for each item.
    *   Prioritized vendor selection based on price, lead time, and quality.
    *   Recommendations for batch ordering or standing orders.
    *   A proposed timeline for procurement.
*   A draft purchase order (in a format specified by the institution).
*   Any relevant safety information (e.g., MSDS links).
*   A summary report outlining the total cost of procurement and any potential cost-saving opportunities.
*   Alternative sourcing options (if available/applicable).

**Example Interaction:**

**User:** "Here's a protocol for performing a plaque assay to determine the titer of a VSV stock. [Provides the full plaque assay protocol text].  The budget for this experiment is $500."

**Agent:**  (After analyzing the protocol) "Okay, I have analyzed the plaque assay protocol and have identified the following required reagents and supplies..."  (Follows the output format described above, providing a comprehensive list, procurement strategy, and draft purchase order, staying within the $500 budget).

**Key Considerations for Implementation:**

*   **Database Integration:** The success of this system relies heavily on integration with existing inventory management systems and institutional procurement databases.
*   **API Access:** API access to vendor websites would significantly improve the agent's ability to retrieve product information and pricing automatically.
*   **Regular Updates:** The agent's knowledge base (reagents, vendors, product information) needs to be updated regularly to reflect changes in pricing, availability, and product specifications.
*   **Human Oversight:**  While the agent can automate many aspects of the procurement process, human oversight is still essential to review purchase orders, address unexpected issues, and ensure compliance with all relevant policies and regulations.
"""


updated_prompt = pal.refine_prompt(prompt, glyph_refinement=True)
print(updated_prompt)

INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.


<human_instructions>
- Treat each glyph as a direct instruction to be followed sequentially, driving the process to completion.
- Deliver the final result as indicated by the glyph code, omitting any extraneous commentary.
- Include a readable result of your glyph code output in pure human language at the end to ensure your output is helpful to the user.
- Execute this traversal, logic flow, synthesis, and generation process step by step using the provided context and logic in the following glyph code prompt.
</human_instructions>

{
  Φ(Develop a system for categorizing different types of user questions.)
  Θ(The system must be easily understood and applied by human moderators. It should be comprehensive enough to cover a wide range of question types, but not so complex that it becomes difficult to use.  Consider factors like clarity, conciseness, ambiguity, topic, and intent. The output should be a clear set of categories with descriptions.)
  ↹(Focus on question topic, question clar

In [5]:
# Now we can create a new Role and add it 
from promptpal.roles import Role
